# Inspection of linear models

In this notebook, we will discuss how to inspect attribute exposed by linear models after fitting and some of the caveats to have in mind when trying to interpret them.

To illustrate this example, we will use the penguins dataset.

In [ ]:
import pandas as pd

data = pd.read_csv("../datasets/penguins.csv")
data.head()

From the dataset, we will select a subset of features that we can easily get insight without to be an expert.

In [ ]:
data = data[[
    "Culmen Length (mm)",
    "Culmen Depth (mm)",
    "Flipper Length (mm)",
    "Body Mass (g)",
    "Sex",
]].dropna(axis="index")

In [ ]:
data.info()

For our problem, we will try to predict the body mass of a penguins given other measurement and whether or not a penguin is a male or female.

In [ ]:
target_name = "Body Mass (g)"
X, y = data.drop(columns=target_name), data[target_name]

In [ ]:
import sklearn
sklearn.set_config(display="diagram")

In this regard, the "Sex" of the penguin is a categorical variable and we will need to encode it. We will use a one-hot encoder for such processing.

In [ ]:
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import make_column_selector as selector
from sklearn.compose import make_column_transformer

preprocessor = make_column_transformer(
    (OneHotEncoder(handle_unknown="ignore"), selector(dtype_include=object)),
    remainder="passthrough",
    verbose_feature_names_out=False,  # to be less verbose in the feature names
)
preprocessor

We will use a `RidgeCV` model regarding the modeling between the features and the target.

In [ ]:
import numpy as np
from sklearn.pipeline import make_pipeline
from sklearn.linear_model import RidgeCV

alphas = np.logspace(-3, 3, num=100)
model = make_pipeline(
    preprocessor, RidgeCV(alphas=alphas)
)
model

## Evaluate your model

Before to do anything, be aware that any "interpretation" will be done on the model trained and not the data generative process. If your model preforms poorly, you can still inspect it. However, the interpretation will not be linked to the original dataset since the model did not succeed to model properly these data.

Thus, an important step is to have an idea of the model accuracy. Let's perform a cross-validation to have an idea on the statistical model performance.

In [ ]:
import pandas as pd
from sklearn.model_selection import cross_validate, RepeatedKFold

cv = RepeatedKFold(n_repeats=30, n_splits=5, random_state=0)
cv_results = cross_validate(
    model, X, y, cv=cv,
    return_estimator=True, return_train_score=True,
    n_jobs=-1,
)
cv_results = pd.DataFrame(cv_results)

In [ ]:
import seaborn as sns
sns.set_context("poster")

In [ ]:
ax = cv_results[["train_score", "test_score"]].plot.hist(bins=30, alpha=0.8)
ax.set_xlim([0, 1])
_ = ax.legend(loc="upper left")

Good news, it seems that we get a reasonable score. Our model is capable of modeling the provided dataset.

Since we stored the models trained and tested on each cross-validation fold, we can now check their coefficients.

<div class="alert alert-success">
    <p><b>EXERCISE</b>:</p>
    Make a box plot of the coefficients the different models.
</div>

In [ ]:
# %load solutions/solution_01.py

By storing those during the cross-validation, we will have an information regarding the variability of these coefficients.

In [ ]:
# %load solutions/solution_02.py

## Sign of the coefficients

A positive coefficient tell us that when their is an increase of the feature value, then their is an increase of the target value.

<div class="alert alert-success">
    <p><b>QUESTION</b>:</p>
    Why is the coefficient associated to <tt>Culmen Depth (mm)</tt> negative? Does the body mass of the penguins decreases with the depth of their culmen?
</div>

The coefficients of a linear model are a conditional association: they quantify the variation of a the output (the body mass) when the given feature is varied, keeping all other features constant. We should not interpret them as a marginal association, characterizing the link between the two quantities ignoring all the rest.

## Effect of correlated features

In the above plot, we can as well think that our current encoding will create two features that are exactly anticorrelated: male and female penguins. This will have an impact on the coefficient as well. You might detect correlated feature by looking at the variation. Indeed, since only one of the feature is required, the model could use one of the two features to model the problem and depending of the CV fold, it would change.

Let's see what would be the effect on removing one of the sex category. In addition, we also have an issue with the `"."` category.

<div class="alert alert-success">
    <p><b>EXERCISE</b>:</p>
    <ul>
        <li>Remove the sample containg the category <tt>"."</tt>.</li>
        <li>Change the preprocessor to drop one of the <tt>"Sex"</tt> column in the <tt>OneHotEncoder</tt>.</li>
        <li>Plot the coefficients.</li>
    </ul>
    What is the effect on the coefficients?
</div>

In [ ]:
# %load solutions/solution_03.py

We see that we have 1 sample. We will remove it from the database.

In [ ]:
# %load solutions/solution_04.py

Now, we will modify our preprocessor such that we drop one of the column.

In [ ]:
# %load solutions/solution_05.py

In [ ]:
# %load solutions/solution_06.py

Let's have first a quick look that our overall results did not drop.

In [ ]:
# %load solutions/solution_07.py

In [ ]:
# %load solutions/solution_08.py

Let's inspect our coefficients now.

In [ ]:
# %load solutions/solution_09.py

In [ ]:
# %load solutions/solution_10.py

We observe that we have much less coefficients variation. In addition, we observe that the correlated features has an impact on all features.

## Scale of coefficients

In [ ]:
coef.mean()

The value of the coefficients are expressed in their original unit. The flipper length coefficient value is expressed in g per mm. Increasing the flipper length of 1 mm will increase the body mass of 39 g. However, if features are not in the same unit, we cannot compare betweem them. One need to rescale the coefficients to have a relative comparison between them.

In [ ]:
X_transformed = pd.DataFrame(
    preprocessor.fit_transform(X),
    columns=preprocessor.get_feature_names_out(X.columns),
)

X_transformed.std(axis=0).plot(kind='barh', figsize=(9, 7))
plt.title('Features std. dev.')

So the right way to do so is to rescale the data during `fit`.

In [ ]:
from sklearn.preprocessing import StandardScaler

preprocessor = make_column_transformer(
    (OneHotEncoder(drop="if_binary", handle_unknown="ignore"), selector(dtype_include=object)),
    (StandardScaler(), selector(dtype_exclude=object)),
    verbose_feature_names_out=False,  # to be less verbose in the feature names
)
preprocessor

In [ ]:
import numpy as np
from sklearn.pipeline import make_pipeline
from sklearn.linear_model import RidgeCV

alphas = np.logspace(-3, 3, num=100)
model = make_pipeline(
    preprocessor, RidgeCV(alphas=alphas)
)
model

In [ ]:
cv_results = cross_validate(
    model, X, y, cv=cv,
    return_estimator=True, return_train_score=True,
    n_jobs=-1,
)
cv_results = pd.DataFrame(cv_results)

In [ ]:
ax = cv_results[["train_score", "test_score"]].plot.hist(bins=30, alpha=0.8)
ax.set_xlim([0, 1])
_ = ax.legend(loc="upper left")

Let's inspect our coefficients now.

In [ ]:
preprocessor.fit(X)
coef = [estimator[-1].coef_ for estimator in cv_results["estimator"]]
coef = pd.DataFrame(coef, columns=preprocessor.get_feature_names_out(X.columns))

In [ ]:
_, ax = plt.subplots(figsize=(8, 6))
ax = sns.boxplot(data=coef, orient='h', ax=ax, color="tab:blue")
ax.set_title("Coefficients during cross-validation")
_ = ax.vlines(0, 0, len(X.columns), linestyle="--", alpha=0.4, color="black")

Now that the coefficients have been scaled, we can safely compare them.

## Impact of the regularization on the coefficients

We already mentioned this specificity earlier. One should check the variation of the regularization parameter in the cross-valdiation as well:

In [ ]:
alpha = pd.Series(
    [estimator[-1].alpha_ for estimator in cv_results["estimator"]]
)
alpha

In [ ]:
alpha.mean(), alpha.std()

Here, we see that we have a little variation and we have a single type of models that work well. However, since regularization will shrink coefficients zeros, you need to ensure to look at this specific aspect. Let's have a look at forcing the regualarization strength in the ridge model.

<div class="alert alert-success">
    <p><b>EXERCISE</b>:</p>
    <ul>
        <li>Run two pipelines by changing <tt>alpha</tt> from 1 to 100,000.</li>
        <li>Plot the coefficients.</li>
    </ul>
    What is the effect of the regularization parameter on the coefficients.
</div>

In [ ]:
# %load solutions/solution_11.py

In [ ]:
# %load solutions/solution_12.py

Let's inspect our coefficients now.

In [ ]:
# %load solutions/solution_13.py

In [ ]:
# %load solutions/solution_14.py

Now, let increase the strength to 100,000.

In [ ]:
# %load solutions/solution_15.py

In [ ]:
# %load solutions/solution_16.py

Let's inspect our coefficients now.

In [ ]:
# %load solutions/solution_17.py

In [ ]:
# %load solutions/solution_18.py

We observe that the coefficients are shrinked.